In [1]:
import torch
import torch.nn as nn
import torch.optim as optim

In [2]:
sentence = "Repeat is the best medicine for memory".split()

In [3]:
vocab = list(set(sentence))
print(vocab)

['is', 'memory', 'best', 'for', 'the', 'Repeat', 'medicine']


In [4]:
word2index = {tkn : i for i, tkn in enumerate(vocab, 1)}
word2index['<unk>'] = 0

In [5]:
print(word2index)

{'is': 1, 'memory': 2, 'best': 3, 'for': 4, 'the': 5, 'Repeat': 6, 'medicine': 7, '<unk>': 0}


In [6]:
index2word = {v: k for k, v in word2index.items()}
print(index2word)

{1: 'is', 2: 'memory', 3: 'best', 4: 'for', 5: 'the', 6: 'Repeat', 7: 'medicine', 0: '<unk>'}


In [7]:
def build_data(sentence, word2index):
    encoded = [word2index[token] for token in sentence] # 각 문자를 정수로 변환. 
    input_seq, label_seq = encoded[:-1], encoded[1:] # 입력 시퀀스와 레이블 시퀀스를 분리
    input_seq = torch.LongTensor(input_seq).unsqueeze(0) # 배치 차원 추가
    label_seq = torch.LongTensor(label_seq).unsqueeze(0) # 배치 차원 추가
    return input_seq, label_seq

In [8]:
X, Y = build_data(sentence, word2index)

In [9]:
print(X)
print(Y)

tensor([[6, 1, 5, 3, 7, 4]])
tensor([[1, 5, 3, 7, 4, 2]])


In [10]:
class Net(nn.Module):
    def __init__(self, vocab_size, input_size, hidden_size, batch_first=True):
        super(Net, self).__init__()
        self.embedding_layer = nn.Embedding(num_embeddings=vocab_size, # 워드 임베딩
                                            embedding_dim=input_size)
        self.rnn_layer = nn.RNN(input_size, hidden_size, # 입력 차원, 은닉 상태의 크기 정의
                                batch_first=batch_first)
        self.linear = nn.Linear(hidden_size, vocab_size) # 출력은 원-핫 벡터의 크기를 가져야함. 또는 단어 집합의 크기만큼 가져야함.

    def forward(self, x):
        # 1. 임베딩 층
        # 크기변화: (배치 크기, 시퀀스 길이) => (배치 크기, 시퀀스 길이, 임베딩 차원)
        output = self.embedding_layer(x)
        # 2. RNN 층
        # 크기변화: (배치 크기, 시퀀스 길이, 임베딩 차원)
        # => output (배치 크기, 시퀀스 길이, 은닉층 크기), hidden (1, 배치 크기, 은닉층 크기)
        output, hidden = self.rnn_layer(output)
        # 3. 최종 출력층
        # 크기변화: (배치 크기, 시퀀스 길이, 은닉층 크기) => (배치 크기, 시퀀스 길이, 단어장 크기)
        output = self.linear(output)
        # 4. view를 통해서 배치 차원 제거
        # 크기변화: (배치 크기, 시퀀스 길이, 단어장 크기) => (배치 크기*시퀀스 길이, 단어장 크기)
        return output.view(-1, output.size(2))

In [11]:
vocab_size = len(word2index)  # 단어장의 크기는 임베딩 층, 최종 출력층에 사용된다. <unk> 토큰을 크기에 포함한다.
input_size = 5  # 임베딩 된 차원의 크기 및 RNN 층 입력 차원의 크기
hidden_size = 20  # RNN의 은닉층 크기

In [12]:
# 모델 생성
model = Net(vocab_size, input_size, hidden_size, batch_first=True)
# 손실함수 정의
loss_function = nn.CrossEntropyLoss() # 소프트맥스 함수 포함이며 실제값은 원-핫 인코딩 안 해도 됨.
# 옵티마이저 정의
optimizer = optim.Adam(params=model.parameters())

In [13]:
output = model(X)
print(output)

tensor([[-0.2279,  0.1570,  0.2473, -0.2115, -0.1381, -0.1073,  0.1536,  0.0634],
        [-0.1824,  0.2138,  0.2816,  0.0256, -0.2844, -0.4099,  0.2268, -0.2251],
        [-0.1424,  0.1909,  0.0026, -0.1331,  0.0170,  0.1587, -0.0158, -0.1792],
        [-0.0573,  0.3855,  0.3729, -0.0909, -0.1450, -0.1756,  0.1259, -0.3016],
        [-0.2166,  0.1024,  0.1393, -0.0935, -0.1702, -0.1547,  0.0966, -0.2260],
        [-0.4083,  0.1789,  0.1023, -0.1955, -0.0690, -0.0283,  0.1423,  0.1071]],
       grad_fn=<ViewBackward>)


In [14]:
decode = lambda y: [index2word.get(x) for x in y]

In [15]:
for step in range(201):
    # 경사 초기화
    optimizer.zero_grad()
    # 순방향 전파
    output = model(X)
    # 손실값 계산
    loss = loss_function(output, Y.view(-1))
    # 역방향 전파
    loss.backward()
    # 매개변수 업데이트
    optimizer.step()
    # 기록
    if step % 40 == 0:
        print("[{:02d}/201] {:.4f} ".format(step+1, loss))
        pred = output.softmax(-1).argmax(-1).tolist()
        print(" ".join(["Repeat"] + decode(pred)))
        print()

[01/201] 2.2009 
Repeat memory memory is is memory is

[41/201] 1.6134 
Repeat memory memory best medicine for memory

[81/201] 0.9258 
Repeat is the best medicine for memory

[121/201] 0.4794 
Repeat is the best medicine for memory

[161/201] 0.2650 
Repeat is the best medicine for memory

[201/201] 0.1607 
Repeat is the best medicine for memory

